This notebook follows the tutorial found at:
https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/


In [2]:
import mailparser
import sys
from os import listdir
from os.path import isfile, join
import pandas as pd
import email
import numpy as np
from talon.signature.bruteforce import extract_signature
import nltk
from nltk.tokenize import sent_tokenize
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

#one time executions
#nltk.download('punkt')
#nltk.download('stopwords')
#! wget http://nlp.stanford.edu/data/glove.6B.zip
#! unzip glove*.zip


Load dataframe produced by the Data_Wranlging notebook. 

In [3]:
#Load Data
PICKLE_LOC = "../data/dataframes/wrangled_enron_df.pkl"
enron_df = pd.read_pickle(PICKLE_LOC)

Checking earliest and latest date. 

In [4]:
start_date = '2001-10-01 00:00:00'
end_date = '2001-10-14 23:59:59'
summarization_mask = (enron_df['Date'] >= start_date) & (enron_df['Date'] <= end_date)
enron_masked_df = enron_df.loc[summarization_mask]

In [5]:
#This function removes stopwords
#def remove_stopwords(sen):
#    sen_new = " ".join([i for i in sen if i not in stop_words])
#    return sen_new

In [6]:
#Splitting the text in emails into sentences
sentences = enron_masked_df.Extractive_Sentences.tolist()
#flatten list
sentences = [y for x in sentences for y in x]





#Finding length
#sent_len = []
#for s in enron_summary_test['Body']:
#    sentence_token = sent_tokenize(s, language='english')
    #sentences.append(sentence_token)
    #Treat \n tokens as new sentences. 
#    for t in sentence_token:
#        sent_len.append(len(t))
#sentences = [y for x in sentences for y in x]
#Remove duplicate sentences
#sentences = list(set(sentences))

In [7]:
#print(len(sentences))
#print(len(sent_len))
#sentences.sort(key=len, reverse=True)
#print(sentences)
#sent_len.sort(reverse = True)
#print(sent_len)

In [8]:
#Pull out clean tokenized sentences. 
clean_sentences = enron_masked_df.Tokenized_Body.tolist()
#flatten list
clean_sentences = [y for x in clean_sentences for y in x]


In [9]:
#Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [10]:
#Creating vectors for the sentences. 
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [11]:
len(sentence_vectors)

1163

In [12]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)


In [13]:
# Specify number of sentences to form the summary
sn = 30
# Generate summary
for i in range(sn):
  print("Sentence "+ str(i) + ": " + ranked_sentences[i][1])

Sentence 0: Work environment: pay and benefits true meritocracy unlimited opportunities The results of our survey confirmed for the Management Committee and me the great things you feel about Enron and also what you see as the issues we must address to move Enron forward.
Sentence 1: A complete story on the origin of Halloween will be on the Business Cafe Web site October 2531 at http://www.BusinessCafeOnline.com Your October 2001 issue of Elert for personal development in business is now available on http://www.BusinessCafeOnline.com Included in the October 2001 issue are these three articles: Flying Off Over Office Politics What Men Can Learn
Sentence 2: full story Previous Saturday's interviews THEMES PDVSA's Petroleum Education program Story Previous Themes SOCIAL ROLE OF BUSINESS CITGO: Donate $1 million worth of fuel for disaster relief Sept 21 Full story Previous stories ENVIROMENT Petrozuata preseted results on clean air in its operational areas Julio Rodr?guez, gerente de (ASH

In [15]:
#ranked_sentences
#enron_summary_test['Body']